This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([177.28302038, 150.37102595, 202.0924573 , ...,  47.20790294,
        23.809066  ,   0.        ]), array([ 88.64152247,  75.1855248 , 101.04624221, ...,  23.60395147,
        11.904533  ,   0.        ]), OrderedDict([('R2', 0.9971847508089243), ('root_mean_squared_error', 70.04066231714356), ('mean_absolute_error', 52.21429446970149), ('rmse_over_stdev', 0.7464900773521139)]), None)
test_quad = (array([325.49788774, 218.80057061, 214.18575236, 182.12903292,
       121.20205853, 114.1942393 , 120.56546405, 156.51523   ,
       110.503751  ,  95.94694389, 214.25291566, 202.29096841,
       200.04746941, 181.65974631,  71.54661553, 130.71079563,
        99.38751557, 188.72532304, 146.05982872, 191.51161513,
       166.69938886, 179.99544416, 142.48339282, 229.72245207,
        54.94979021,  59.67920269, 114.36792625,  70.6664565 ,
        77.04502053,  91.15693652,  88.46084748,  74.58551681,
       165.79365236, 156.94124086, 168.46495631, 121.20443823,
        58.99391702, 109.47390069,   0.        ,  24.184234  ,
       121.57100715, 183.35068747, 390.4260105 ,  98.51156618,
       254.26505048,  86.04966315,  60.35669252,  89.72129861,
        73.92834539,  80.73776665,  89.50296147, 103.80111055,
       134.18069341,   8.216444  ,  71.44563177,  32.0094037 ,
       179.25872313,   0.        ,  49.34328566,   5.90096117,
        36.36319225,  27.31353641,   0.        , 141.66475147,
        64.50496061, 101.79799033, 128.57645296, 261.7248035 ,
        19.46088327,  29.16320729, 130.86141575, 236.69985867,
        88.18349836, 206.15823625, 119.77883703, 151.79744052,
       289.7129778 ,  59.63498938,  70.83577191,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,  94.50620971,  85.96097361,  89.23565059,
        26.82825386,   0.        ,   0.        ,   0.        ,
         7.74558848,  25.17480383,  53.84255407,   0.        ,
         0.        ,  40.83543394,  43.48823119,  36.50751242,
        43.12521542,   0.        ,  19.2047597 , 161.61585578,
       108.97985761,  14.09444405,  87.36071311,   0.        ,
         0.        ,   0.        ,   0.        ,  11.93148749,
         0.        ,  16.20435918,   8.97062587,  35.50340569,
         7.37896699,  88.38029969,  74.7468692 , 117.12577837,
        62.94422008,  37.34883385, 122.03873942,  33.20006813,
         0.        ,  57.49950031,  61.21370263,  49.15732575,
        28.2257425 ,   0.        ,  86.88965402, 137.40871252,
        53.54285953,  80.45910798,  65.29481037,  58.6508558 ,
        85.23364294, 126.88631207, 468.0346095 , 138.89285472,
        21.17208738,   0.        , 122.31135567,   0.        ,
       130.59739217, 166.18742887, 285.94816437, 106.70233503,
       298.36883132, 207.51571353,  39.9617942 ,  47.36934588,
       197.24625694, 175.56939781, 321.98957475, 367.6244316 ,
       211.47682381, 113.90148661,  48.04117053,  47.45866309,
        35.07688959,  64.97491856,  28.05755061,   9.106651  ,
         0.        ,  10.51108275,   0.        ,   0.        ,
        21.5174731 ,   0.        ,  58.88531967,  72.01080223,
        64.51928629,  66.64638295,  33.27336669,  39.69329769,
        45.2879705 , 169.71976527, 224.40216532, 138.75817334,
       122.98024015, 124.72986509, 152.42942927,  94.65388494,
        83.02690466,   0.        ,   0.        , 141.37372586,
        43.12872844,  25.08521475,  36.35679348,  51.75595534,
        47.24337609, 110.71694286,  23.6410205 ,  44.01037784,
        54.82773069,  57.60884947,  95.49667394,  49.42529029,
        19.80860463,  17.71877028, 125.21536969,  32.611025  ,
        21.12324531,   0.        ,   0.        ,   0.        ,
         5.20671817,   0.        , 129.60153223, 159.11770491,
       191.11304901, 217.25282394, 209.0198447 , 361.57977723,
       106.514784  , 134.67311657,  81.57468903, 129.60777958,
       136.29376713, 111.92047078, 244.14891128, 189.82462955,
       124.31412396, 235.40922492, 192.28465626, 220.16151104,
       196.40913358, 122.67765683, 141.5352297 , 133.95472581,
        87.04599469, 129.34261745, 210.33572768, 103.16515278,
       319.11663064, 292.53608379, 351.81750188, 367.1832279 ,
       237.06234447, 362.46105447, 294.31041312, 177.34615626,
       323.82263694, 232.69981876, 199.00234553, 143.51720548,
       130.55899831, 210.10783532, 212.75663082, 252.16135397,
       184.51653892, 103.45613548,  98.39446077, 187.88525488,
       192.71275543, 197.30900025, 116.27871809, 154.70761089,
       364.70710132, 245.48897712, 338.95548037, 186.71778892,
       230.9176424 , 346.08790771, 142.82619229,  84.93486971,
       144.15449831, 182.22098097, 213.36565687,  64.02417181,
       139.23135132,  54.1587672 ,  61.57636317, 126.81817022,
        45.29321414,  69.20850033,  83.15072959, 184.90772539,
       199.57809553, 233.38366567, 193.51782743, 283.04242932,
       184.42082504, 132.21158822, 204.55489376, 845.33730031,
       950.22790588, 209.325439  , 120.15829999,  67.71649743,
        46.57820663,  82.82809965,  92.79832923, 130.56858186,
       106.91311671, 125.31262584,  36.00668628,  18.90387551,
        12.23389363,  33.14817606,  42.88927876,  29.54783309,
        48.8429076 ,  77.59338173, 161.56318378, 234.93787541,
       147.33684207, 102.11231532, 124.01781744,  95.20604545,
       173.38559503, 174.90906864, 197.17633245, 188.27143603,
       198.01350063, 146.55208232, 182.10184863, 172.75454393,
       146.5521539 , 111.38413582, 181.38757478, 220.40870164,
       189.99865166,  54.91523627,   0.        ,  30.77112483,
       123.93596706, 159.40354586, 114.75434014,  86.69975106,
        97.91858002, 137.58344095, 171.49036331, 208.03565815,
       111.06672169, 117.14316894,  98.78441841, 145.31046867,
       335.32129343, 163.49712402,  89.81673775,  87.6081324 ,
        56.51132963,  66.99094092,  25.29305299,  20.06194164,
        95.30504358,  37.32259412,  95.04900638,  72.94226156,
       113.17048098,  95.6649845 ,  50.44715644,  71.47961918,
        49.79361192,  28.44172814,  35.04593958, 105.85969748,
       143.07547478, 140.73746488,  52.87341873,  87.08840286,
       101.02067384, 129.16762848,  35.07038698,  95.53083323,
         0.        ,  62.09475103,  98.25644159,   0.        ,
       112.1749515 , 124.75938063, 198.10543586, 136.25298986,
        62.32476733,  56.31928935, 160.33463254, 121.97489618,
        74.35889627,  51.91571301,  21.35392387,   9.31924847,
        93.06800775,  21.53240718,  12.40117081,  11.01778613,
         0.        ,  14.2008055 ,  34.10036925,   0.        ,
        49.2018258 ,  29.90345073,   0.        ,  58.14981868,
       139.06215854,  75.6574945 ,  76.84371871,  82.90154417,
       221.76800698,  40.70021975,  52.2740325 ,  70.45416536,
        70.59869671,  65.52025943,  97.23119343,  39.11088786,
         0.        ,  46.95075044,  56.66564979,  87.26704216,
       123.23318414,  37.60933861,   0.        ,  34.86837846,
        80.17227524,  37.42219894,  24.51250794,  66.50124863,
        53.68625362, 173.11303813,  33.00480049,   0.        ,
       289.029515  ,   0.        ,   0.        ,  36.55454693,
        19.41019935]), array([ 4.02313702e-005,  9.37558144e-249,  1.25837471e-270,
        1.73261865e-122,  2.78014917e-028,  5.31157834e-192,
        9.64563318e-028,  8.22558799e-135,  1.93841233e-132,
        2.16669030e-231,  8.06501136e-192,  4.30335593e-231,
        0.00000000e+000,  1.19131997e-115,  1.89507035e-027,
        5.15477689e-192,  1.13994844e-027,  1.15853867e-239,
        2.02031130e-010,  1.45502867e-010,  7.70558633e-236,
        0.00000000e+000,  0.00000000e+000,  3.07381952e-304,
        2.32522385e-032,  0.00000000e+000,  3.77609473e-133,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  1.95964481e-238,
        1.02497281e-132,  1.04159795e-305,  0.00000000e+000,
        1.13959437e-119,  6.57112397e-255,  6.58045386e-003,
        2.28149910e-156,  1.00598781e-038,  0.00000000e+000,
        9.20767117e-137,  3.16365228e-025,  2.89213462e-115,
        2.35402083e+000,  1.77066025e-020,  1.77195334e+000,
        3.56856916e-022,  2.64374473e+000,  7.19480505e-004,
        2.56373178e-006,  4.20480533e-002,  0.00000000e+000,
        1.31512880e-013,  0.00000000e+000,  3.16631105e-009,
        0.00000000e+000,  2.45215831e-098,  1.35588136e-075,
        5.43915884e-027,  7.13890521e-039,  0.00000000e+000,
        0.00000000e+000,  1.44440013e-109,  8.48916040e-008,
        1.96357114e-009,  1.96058788e-009,  1.28384567e-002,
        1.57577633e-004,  1.01200349e-008,  1.24453710e+001,
        2.64618467e-131,  5.30510007e-045,  2.83400967e-006,
        4.64821971e-002,  0.00000000e+000,  4.85354782e-084,
        2.24570524e-010,  0.00000000e+000, -7.18870662e-239,
        0.00000000e+000,  7.10536500e-239,  1.42774333e-236,
        0.00000000e+000,  7.02025822e-084,  3.03437115e-241,
        1.14729682e-237,  2.32323209e-012,  2.83541773e-007,
        8.60476045e-070,  3.31623371e-242,  2.58578678e-007,
        1.97154806e-012,  2.28683121e-010,  0.00000000e+000,
        0.00000000e+000,  6.91682874e-001,  2.56210901e-006,
        9.42080114e-002,  9.01557907e-003,  0.00000000e+000,
        6.97331495e-001,  1.31411899e-011,  1.83413988e-001,
        1.74988923e-084,  2.32373964e-095,  6.87664887e-014,
        2.13010632e-241,  5.26796993e-238,  3.54582929e-239,
        3.63847095e-072,  4.97738295e-239,  8.93664713e-104,
        0.00000000e+000,  7.01165100e-001,  0.00000000e+000,
        6.91097764e-009,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  1.34105070e-016,  1.18100064e-006,
        1.66807944e-095,  0.00000000e+000,  1.95482077e-031,
        1.03401516e-013,  7.05334893e-003,  4.42589354e-013,
        0.00000000e+000,  2.29150661e-003,  2.52853386e-161,
        8.69064282e-024,  9.73511720e-287,  0.00000000e+000,
        0.00000000e+000,  2.04434208e-026,  3.31692117e-011,
        1.65482939e-024,  6.43377347e-097,  8.57839615e-006,
        0.00000000e+000,  6.11556778e+001,  0.00000000e+000,
        0.00000000e+000,  2.16403275e-011,  1.58341565e-024,
        8.70887330e-023,  0.00000000e+000,  2.43180886e-027,
        2.68540700e-105,  3.25858152e-002,  2.32545494e-013,
        1.38785794e-001,  3.79290276e-160,  1.31608073e-031,
        3.69906264e-005,  4.87473896e-088,  1.74271768e-006,
        7.41769368e-008,  8.00404375e-004,  3.90339183e-077,
        6.98393230e-002,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000, -5.73417739e-011,  0.00000000e+000,
        2.68165322e-013,  2.21076095e-023,  6.08502569e-088,
        9.49619697e-102,  1.87760118e+001,  1.00563893e-028,
        6.34133841e-102,  9.95954643e+000,  7.94246911e-002,
        8.30657550e-236,  1.99312823e-233,  5.16426216e-006,
        3.50186598e-232,  2.59950034e-233,  1.99852667e-076,
        6.65103399e-053,  2.21457611e-023,  0.00000000e+000,
        0.00000000e+000,  1.38971553e-101,  1.24114589e-006,
        6.14916987e+000,  2.18096121e-017,  7.64842006e-008,
        2.84208486e-025,  1.44323164e-103,  0.00000000e+000,
        5.28136698e-232,  3.92830465e-233,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  1.34547739e-020,
       -1.15917645e-066,  1.00534419e-006,  0.00000000e+000,
        1.47933808e-102,  0.00000000e+000,  1.57196043e-004,
        3.30828465e-030,  3.29865207e-013,  0.00000000e+000,
        2.73855791e-001,  1.44341356e-026,  1.04649317e-014,
        5.83408882e+000,  0.00000000e+000,  8.03369001e-001,
        1.76082854e-001,  2.46596556e-001,  1.15832060e+001,
        2.86532724e+000,  3.71801574e+000,  2.20257399e-001,
        2.87503449e-001,  3.65494542e-001,  3.18495170e-001,
        4.67026614e-001,  2.44679438e-008,  5.52259806e-002,
        3.69046743e-001,  1.94025744e-001,  2.78393283e-001,
        1.42608912e-008,  8.37657996e+000,  1.23474861e+001,
        5.14130895e+000,  2.63042658e+000,  1.06054768e-003,
        1.12729954e-003,  7.58154540e-001,  5.07063730e-020,
        5.01701368e-001,  3.67586416e-008,  7.27582695e-002,
        1.21814745e-011,  8.93722558e+000,  5.06977543e+000,
        7.88686199e-004,  3.78911703e+000,  2.87732110e-001,
        1.58237495e-001,  5.25784929e-001,  1.86510952e-001,
        2.61499994e-008,  1.14827447e-020,  1.35954353e-008,
        2.12045236e-008,  1.93647474e+001,  4.81250224e+000,
        2.28602310e+000,  4.16360120e-004,  7.53021801e-001,
        5.50267834e-001,  7.50068168e-001,  1.30158621e-011,
        7.26842899e+000,  7.77850482e+000,  3.43204581e+000,
        2.07424369e+000,  2.88628801e-001,  3.16023202e-001,
        3.81383046e-001,  1.24167910e-001,  2.92262410e-001,
        1.42564681e-008,  5.73357591e+000,  1.35879497e+001,
        5.35805827e+000,  4.01329164e-011,  2.30339091e-004,
        3.67879237e+000,  3.45785900e-004,  1.65144398e-010,
        3.78962305e-001,  5.79245411e-001,  4.29921053e-001,
        3.39523674e-002,  4.02123867e-011,  3.12000558e-010,
        6.97067524e-010,  5.42324939e-071,  2.63574150e-001,
        3.71481192e-017,  6.67144657e-071,  6.48694105e-004,
        6.48022347e-004,  1.11754315e-003,  2.93454217e-001,
        1.84151016e-045,  5.94840739e-006,  2.73615895e-001,
        1.58845228e-017,  7.00909933e-011,  4.52619153e-004,
        3.33501211e-004,  6.25711651e-001,  4.40714115e-004,
        5.27055317e-015,  7.81974164e-051,  5.40899500e-016,
        2.46687204e-103,  7.87633579e-039,  1.19999397e-290,
        1.17782742e-144,  1.43220593e-063,  8.18244616e-051,
        1.71499976e-037,  6.07102308e-016,  6.40353629e-025,
        1.49288881e-003,  1.44184006e-003,  1.24237521e-003,
        9.51452316e-004,  1.32399854e-003,  1.83998550e-003,
        1.28395543e-003,  3.22117832e-028,  2.33593943e-114,
        2.89299184e-028,  6.19849308e-004,  0.00000000e+000,
        3.96457700e+000,  7.24697835e-002,  1.05191627e+001,
        6.38236153e+000,  1.87941006e+001,  6.85246808e+000,
        0.00000000e+000,  2.62238869e+001,  8.21446923e-002,
        0.00000000e+000,  2.47269338e-004,  5.51798242e+000,
        4.49083689e+001,  2.15181131e+000,  6.66309037e-001,
        1.83083577e-012,  4.36862383e-020,  2.67162943e-020,
        5.74225297e-082,  1.65322448e-015,  4.59970137e-036,
        1.55963586e-002,  3.58698787e-013,  8.56302146e-003,
        9.08039529e-030,  9.08518603e-013,  4.35657698e-004,
        2.13322183e-021,  3.77042145e-087,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  3.12802855e-005,
        0.00000000e+000,  0.00000000e+000,  6.20177897e-105,
        0.00000000e+000,  6.96431159e-024,  0.00000000e+000,
        3.93334951e-081,  2.06820834e-035,  0.00000000e+000,
        3.26772798e-073,  3.39376810e-005,  2.10666454e-053,
        2.89590935e-080,  7.77815139e-078,  8.03943158e-031,
        2.11072465e-011,  7.85705854e-005,  7.64297079e-002,
        1.30655194e-001,  3.76290045e-028,  6.72251429e-019,
        3.80016938e-063,  1.01022438e-001,  3.80446792e-031,
        6.52220979e-014,  0.00000000e+000,  0.00000000e+000,
        3.58269641e-028,  0.00000000e+000,  4.39300493e-063,
        6.74008243e-001,  0.00000000e+000,  1.07859130e+000,
        1.72337890e-077,  1.07929430e-027,  1.12937926e-010,
        2.03552237e-017,  9.46494656e-063,  3.06690305e-025,
        5.73128770e-006,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  3.99231470e-012,  3.16637026e-078,
        0.00000000e+000,  7.47906486e-031,  2.30743205e+000,
        1.17299060e-030,  9.79914809e-063,  1.31144371e-022,
        0.00000000e+000,  5.43540001e-028,  7.51878597e-063,
        3.21414554e-017,  1.69994952e-037,  1.78087534e-080,
        7.11685326e-031,  6.88498463e-007,  1.83004321e-080,
        0.00000000e+000,  2.84816338e-014,  0.00000000e+000,
        3.63540670e-013,  2.12448279e-003,  4.32668725e-015]), OrderedDict([('R2', 0.0031612812660493095), ('root_mean_squared_error', 148.35007306490232), ('mean_absolute_error', 108.44355147863203), ('rmse_over_stdev', 1.5811080856998816)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge/RepeatedKFold/split_4'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
